<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/Assignmnet_5_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


# **Team Members**

Zilal Eiz Aldin (Online MCS) Zelalae2@illinois.edu

Mesay Taye (Online MCS) mesayst2@illinois.edu

Jianci Zhai (Online MCS) jianciz2@illinois.edu

# **Contribution of each member**
Hyperparameter Search and Final Check By Mesay

Designing the SGD Algorithm and Testing Pegasos Algorithm  By Zilal

Debugging / Prediction / Testing By Jianci

https://campuswire.com/c/GB46E5679/feed/834

https://liangfgithub.github.io/Coding/F24_Coding5.pdf

##**Part 1 -- By Zilal**
- Design SVM From Scratch


In [2]:
import numpy as np

class LinearSVM:
    def __init__(self, lambda_param, epochs):
       # self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.epochs = epochs

    def fit(self, X, y):
        # Initialize parameters
        self.w =  np.zeros(X.shape[1])  # Random initialization of weights
        self.b = 0  # Bias initialized to 0
        self.t = 0  # Iteration counter

        # Stochastic Gradient Descent
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                xi = X.loc[i, :]
                yi = y[i]
                self.t += 1

                # Compute the hinge loss gradient
                if yi * (np.dot(xi, self.w) + self.b) < 1:
                    dw = self.lambda_param*self.w - yi * xi
                    db = -yi
                else:
                    dw = self.lambda_param*self.w
                    db = 0

                # Add regularization term (derivative of lambda * ||w||^2)
               # dw += 2 * self.lambda_param * self.w

                # Update parameters
                learning_rate  = 1 / (self.lambda_param*self.t)
                self.w -= learning_rate * dw
                self.b -= learning_rate * db

            # Optionally, print the loss after each epoch to track progress
            loss = self.compute_loss(X, y)
            print(f"Epoch {epoch+1}/{self.epochs}, Loss: {loss}")

    def compute_loss(self, X, y):
        # Compute the hinge loss + regularization
        margins = 1 - y * (np.dot(X, self.w) + self.b)
        hinge_loss = np.sum(np.maximum(0, margins)) / X.shape[0]
        reg_loss = self.lambda_param * np.sum(self.w ** 2)
        return hinge_loss + reg_loss

    def predict(self, X):
        # Predict the class labels: 1 if positive, -1 if negative
        return np.sign(np.dot(X, self.w) + self.b)


# **Jianci - Testing SVM**

In [3]:
##Testing
# Example usage:
train_org = pd.read_csv('https://liangfgithub.github.io/Data/coding5_train.csv')
test =  pd.read_csv('https://liangfgithub.github.io/Data/coding5_test.csv')




In [7]:
lambda_param_range = np.linspace(0.001, 4, 500)

In [11]:
# Split the training data into training and validation sets

train =train_org.sample(frac=1, random_state=42).reset_index(drop=True)

# Map Y values: 5 -> -1, 6 -> 1
Y_train=(train['Y']>5).astype(int)*2-1
X_train=train.drop(['Y'] , axis=1)

Y_test=(test['Y']>5).astype(int)*2-1
X_test=test.drop(['Y'] , axis=1)


X_train_split, X_val, Y_train_split, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

# Reset indices to ensure sequential integer indexing
X_train_split = X_train_split.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
Y_train_split = pd.Series(Y_train_split).reset_index(drop=True)
Y_val = pd.Series(Y_val).reset_index(drop=True)

print(f"Training Split Size: {X_train_split.shape[0]} samples")
print(f"Validation Set Size: {X_val.shape[0]} samples\n")


Training Split Size: 160 samples
Validation Set Size: 40 samples



In [12]:
def hyperparameter_search(X_train, Y_train, X_val, Y_val, lambda_values, epochs=50, early_stopping_threshold=0.10):
    best_lambda = None
    best_val_error = float('inf')
    best_model = None

    for idx, lambda_val in enumerate(lambda_values, 1):
        print(f"Lambda {idx}/{len(lambda_values)}: {lambda_val:.6f}")
        model = LinearSVM(lambda_param=lambda_val, epochs=epochs)
        model.fit(X_train, Y_train)

        predictions = model.predict(X_val)
        val_error = np.mean(predictions != Y_val)
        print(f"Validation Error Rate: {val_error:.2%}\n")

        if val_error < best_val_error:
            best_val_error = val_error
            best_lambda = lambda_val
            best_model = model

            if best_val_error < early_stopping_threshold:
                print(f"Desired validation error rate achieved ({best_val_error:.2%}). Stopping search.\n")
                break

    return best_lambda, best_model, best_val_error


In [13]:
# Define the range of lambda values to search
lambda_param_range = np.linspace(0.001, 4, 50)  # 50 values from 0.001 to 4

# Perform hyperparameter search
print("Starting Hyperparameter Search...\n")
best_lambda, best_model, best_val_error = hyperparameter_search(
    X_train_split, Y_train_split, X_val, Y_val,
    lambda_values=lambda_param_range, epochs=50, early_stopping_threshold=0.10
)

print(f"Best Lambda Found: {best_lambda:.6f} with Validation Error Rate: {best_val_error:.2%}\n")


Starting Hyperparameter Search...

Lambda 1/50: 0.001000
Epoch 1/50, Loss: 129.2002355854567
Epoch 2/50, Loss: 35.43506422604813
Epoch 3/50, Loss: 22.464804432947833
Epoch 4/50, Loss: 14.157114893966028
Epoch 5/50, Loss: 10.314252618749997
Epoch 6/50, Loss: 8.202339571946542
Epoch 7/50, Loss: 6.676972557873455
Epoch 8/50, Loss: 5.834465638309788
Epoch 9/50, Loss: 5.45923059733295
Epoch 10/50, Loss: 5.056230615234376
Epoch 11/50, Loss: 4.917874410188533
Epoch 12/50, Loss: 4.680003559570312
Epoch 13/50, Loss: 4.50667512943787
Epoch 14/50, Loss: 3.8858576371173483
Epoch 15/50, Loss: 3.8201806147460577
Epoch 16/50, Loss: 3.7702208206176766
Epoch 17/50, Loss: 3.7248052103157443
Epoch 18/50, Loss: 3.6788862070794752
Epoch 19/50, Loss: 3.319003589842991
Epoch 20/50, Loss: 3.6775267499579956
Epoch 21/50, Loss: 3.3000743545795905
Epoch 22/50, Loss: 3.288395327995869
Epoch 23/50, Loss: 3.082874588346705
Epoch 24/50, Loss: 3.1409736278077087
Epoch 25/50, Loss: 3.304965961375
Epoch 26/50, Loss: 3.

In [14]:

lambda_param = lambda_param_range[1]
print(f"lambda ={lambda_param}")
model = LinearSVM(lambda_param= best_lambda, epochs=50)
model.fit(X_train, Y_train)

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)
train_error_rate = 1-(train_preds == Y_train).sum()/len(Y_train)
test_error_rate = 1-(test_preds == Y_test).sum()/len(Y_test)

cm_train = confusion_matrix(Y_train, train_preds)
cm_test = confusion_matrix(Y_test, test_preds)
print(f"Error Rate on Training: {train_error_rate}")
print(f"Error Rate on Testing: {test_error_rate} around {(test_error_rate * 100).astype(int)}%")
print("-------------------------------------------")
print("-------------------------------------------")
print(f"Confusion Matrix on Training:\n{cm_train}")
print(f"Confusion Matrix on Testing:\n{cm_test}")

lambda =0.08261224489795918
Epoch 1/50, Loss: 1.3089109791173328
Epoch 2/50, Loss: 0.5150690846779249
Epoch 3/50, Loss: 0.4251636029666781
Epoch 4/50, Loss: 0.26920099860564434
Epoch 5/50, Loss: 0.2460674874507089
Epoch 6/50, Loss: 0.22857798492426923
Epoch 7/50, Loss: 0.22740338888436845
Epoch 8/50, Loss: 0.18973222136399429
Epoch 9/50, Loss: 0.1883864274896126
Epoch 10/50, Loss: 0.18611390836955594
Epoch 11/50, Loss: 0.1638422678995647
Epoch 12/50, Loss: 0.16765080748721606
Epoch 13/50, Loss: 0.16994599243829484
Epoch 14/50, Loss: 0.15703984415760866
Epoch 15/50, Loss: 0.1574489098131862
Epoch 16/50, Loss: 0.1567870442894313
Epoch 17/50, Loss: 0.15984983689257107
Epoch 18/50, Loss: 0.14844483043356263
Epoch 19/50, Loss: 0.1475796164032881
Epoch 20/50, Loss: 0.14295600871990286
Epoch 21/50, Loss: 0.14260217610904388
Epoch 22/50, Loss: 0.14436897284021602
Epoch 23/50, Loss: 0.14202705978001354
Epoch 24/50, Loss: 0.14203732146922804
Epoch 25/50, Loss: 0.13897213826779348
Epoch 26/50, Lo

In [ ]:
epochs =50

for lambda_param in lambda_param_range:

  #random shuffle the training data
  train =train_org.sample(frac=1, random_state=42).reset_index(drop=True)


  Y=(train['Y']>5).astype(int)*2-1
  X_train=train.drop(['Y'] , axis=1)


  Y_test=(test['Y']>5).astype(int)*2-1
  X_test=test.drop(['Y'] , axis=1)

  print(f"lambda ={lambda_param}")

  model = LinearSVM(lambda_param=lambda_param, epochs=epochs)
  model.fit(X_train, Y)
  predictions = model.predict(X_test)
  error_rate = 1-(predictions == Y_test).sum()/len(Y_test)
  cm = confusion_matrix(Y_test, predictions)
  print(f"Error Rate: {error_rate}")
  print(f"Confusion Matrix:\n{cm}")


lambda =0.001
Epoch 1/50, Loss: 90.5829658809895
Epoch 2/50, Loss: 45.806166577792865
Epoch 3/50, Loss: 24.784637135161862
Epoch 4/50, Loss: 19.306996022723006
Epoch 5/50, Loss: 12.21402315714317
Epoch 6/50, Loss: 12.513714007520742
Epoch 7/50, Loss: 9.769753738785031
Epoch 8/50, Loss: 8.80670270208666
Epoch 9/50, Loss: 8.561151372530865
Epoch 10/50, Loss: 9.233684634526549
Epoch 11/50, Loss: 8.979964907903879
Epoch 12/50, Loss: 7.4789095394570335
Epoch 13/50, Loss: 8.128133391784749
Epoch 14/50, Loss: 7.500265950637756
Epoch 15/50, Loss: 7.036973038555557
Epoch 16/50, Loss: 7.135635392382814
Epoch 17/50, Loss: 7.298909520630416
Epoch 18/50, Loss: 7.379873828371437
Epoch 19/50, Loss: 6.69396499117242
Epoch 20/50, Loss: 7.171662364655267
Epoch 21/50, Loss: 6.806436795918367
Epoch 22/50, Loss: 6.570479667042012
Epoch 23/50, Loss: 6.636187721880909
Epoch 24/50, Loss: 6.801396340240111
Epoch 25/50, Loss: 6.842346229287377
Epoch 26/50, Loss: 6.68683119389946
Epoch 27/50, Loss: 6.69566210733